In [38]:
import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor,transforms

# print("Torch Version:", torch.__version__)
# print("Torchvision Version:", torchvision.__version__)


In [39]:
trainig_data = datasets.MNIST(
    root = "data",
    train=True,
    download=True,
    transform=ToTensor()

)

test_data = datasets.MNIST(
    root = "data",
    train=False,
    download=True,
    transform=ToTensor()

)

In [40]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])


batch_size = 64
train_dataloader = DataLoader(trainig_data,batch_size=batch_size)
test_dataloader = DataLoader(test_data,batch_size=batch_size)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


In [41]:
# Get cpu, gpu or mps device for training.
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

class Imnet(nn.Module):
 
    def __init__(self):
        super(Imnet, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3, padding=1)  # Garder la même taille en utilisant padding=1
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1)  # Garder la même taille en utilisant padding=1
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.fc1 = nn.Linear(64 * 7 * 7, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self,x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 64 * 7 * 7) 

        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

model = Imnet().to(device)
print(model)


Using cpu device
Imnet(
  (conv1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=3136, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=10, bias=True)
)


In [42]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(),lr=1e-3)

In [43]:
def train(dataloader,model,loss_fn,optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch,(X,y) in enumerate(dataloader):
        X,y = X.to(device),y.to(device) 
        
        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred,y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


In [44]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [45]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.310461  [   64/60000]
loss: 2.300370  [ 6464/60000]
loss: 2.302148  [12864/60000]
loss: 2.297248  [19264/60000]
loss: 2.294058  [25664/60000]
loss: 2.295046  [32064/60000]
loss: 2.302509  [38464/60000]
loss: 2.283924  [44864/60000]
loss: 2.288185  [51264/60000]
loss: 2.293376  [57664/60000]
Test Error: 
 Accuracy: 11.1%, Avg loss: 2.286382 

Epoch 2
-------------------------------
loss: 2.289532  [   64/60000]
loss: 2.281115  [ 6464/60000]
loss: 2.282941  [12864/60000]
loss: 2.272491  [19264/60000]
loss: 2.274923  [25664/60000]
loss: 2.276240  [32064/60000]
loss: 2.273045  [38464/60000]
loss: 2.264050  [44864/60000]
loss: 2.262798  [51264/60000]
loss: 2.259432  [57664/60000]
Test Error: 
 Accuracy: 40.3%, Avg loss: 2.256212 

Epoch 3
-------------------------------
loss: 2.258898  [   64/60000]
loss: 2.247649  [ 6464/60000]
loss: 2.247531  [12864/60000]
loss: 2.231264  [19264/60000]
loss: 2.235343  [25664/60000]
loss: 2.235072  [32064/600